# Bits, bytes and encoded messages

To code our first cipher based on a pseudorandom generator we need to understand a little bit better how information is encoded in python. For this, we coded several functions to help us.

In [1]:
from crypto import bytes_to_bin, bytes_to_hex

message = b"simple message"

bin_repr = bytes_to_bin(message, pre="")
hex_repr = bytes_to_hex(message, pre="")

print(f"message:\n{message}\nlen bytes: {len(message)}\n")
print(f"message in binary:\n{bin_repr}\nlen bits: {len(bin_repr)}\n")
print(f"message in hexadecimal:\n{hex_repr}\nlen hex:{len(hex_repr)}\n")

message:
b'simple message'
len bytes: 14

message in binary:
0111001101101001011011010111000001101100011001010010000001101101011001010111001101110011011000010110011101100101
len bits: 112

message in hexadecimal:
73696d706c65206d657373616765
len hex:28



## Padding bytes (XOR operation)

We need to implement the XOR operation byte-wise, that is how we "pad" our message to convert it to its encrypted form.

In [2]:
from crypto import binary, hexadecimal
import random

a = random.randrange(256)
b = random.randrange(256)

print(f"a: {a} (int), {hexadecimal(a, pre='')} (hex), {binary(a, pre='')} (bin)")
print(f"b: {b} (int), {hexadecimal(b, pre='')} (hex), {binary(b, pre='')} (bin)")

xored = a ^ b
print(f"x: {xored} (int), {hexadecimal(xored, pre='')} (hex), {binary(xored, pre='')} (bin)")

print(f"\n{binary(a, pre='')}\n+\n{binary(b, pre='')}\n=\n{binary(xored, pre='')}")

a: 133 (int), 85 (hex), 10000101 (bin)
b: 37 (int), 25 (hex), 00100101 (bin)
x: 160 (int), a0 (hex), 10100000 (bin)

10000101
+
00100101
=
10100000


## Bytes random generator

We need a pseudorandom genertor for bytes, let's code it!

In Python we have the ```state``` variable to store the state of the pseudorandom generator in ```random``` package.

In [3]:
# in random python pacakge we have the "state" of the PRG
random.seed(10)
state = random.getstate()

print([random.randrange(256) for _ in range(10)])

random.setstate(state)
print([random.randrange(256) for _ in range(20)])

[16, 219, 247, 7, 105, 236, 251, 142, 82, 17]
[16, 219, 247, 7, 105, 236, 251, 142, 82, 17, 250, 167, 38, 127, 184, 22, 215, 71, 181, 195]


In [4]:
import random

random.seed(10)
state = random.getstate()

def PseudoRandomBytes(state: tuple, l: int) -> (bytes, tuple):
    """
    Generates a stream of pseudorandom bytes
    Input:
        - state: a state for the python random pacakge (random.getstate())
        - l: length of the pseudorandom stream of bytes
    Returns:
        - state: the current state of the random
        - bytestream: a bytes class of lenght l
    """
    random.setstate(state)
    prng = []
    
    while len(prng) < l:
        prng.append(hexadecimal(random.randrange(256)))
        
    return random.getstate(), bytes([int(x, 0) for x in prng])

## A class for the parties, let's communicate securely!

In [16]:
class Party:
    def __init__(self, state: tuple):
        self._state = state
        
    def encrypt_decrypt(self, m: bytes) -> bytes:
        new_state, random_bytes = PseudoRandomBytes(self._state, len(m))
        self._state = new_state
        
        return bytes([a ^ b for a, b in zip(m, random_bytes)])

In [17]:
state = random.getstate()

alice = Party(state)
bob = Party(state)

In [18]:
m = b"Hi Bob, how are you doing?"
ctx = alice.encrypt_decrypt(m)
m2 = bob.encrypt_decrypt(ctx)

print(f"message: \n\t{m}\n")
print(f"ciphertext: \n\t{ctx}\n")
print(f"recovered_message: \n\t{m2}")

message: 
	b'Hi Bob, how are you doing?'

ciphertext: 
	b'\xaa<A\xf0\xb2\xb6\xc8]\xe4&,\x1d\xe9\x9b\xfft 5\x80\x91\xc3\xb1\x1blq\x96'

recovered_message: 
	b'Hi Bob, how are you doing?'


In [19]:
m = b"I'm good, thank you!. How do you do?"
ctx = bob.encrypt_decrypt(m)
m2 = alice.encrypt_decrypt(ctx)


print(f"message: \n\t{m}\n")
print(f"ciphertext: \n\t{ctx}\n")
print(f"recovered_message: \n\t{m2}")

message: 
	b"I'm good, thank you!. How do you do?"

ciphertext: 
	b'\xea[E\xa6\x82\xa0>\xa3\xd1[\xff\x9f\x9eN>\xda\x95\xa20\xf6\x9c\xe6\xb2;\x96\x12\xb0}!\x99Nl\x93K"\x08'

recovered_message: 
	b"I'm good, thank you!. How do you do?"
